In [ ]:
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import textdistance

# Configuration initiale de MLflow (facultatif)
# mlflow.set_tracking_uri('votre_uri_de_suivi')

# Lecture des données
df = pd.read_excel("DONNEE_LABEL_FAUX_REGROUPER.xlsx")
df2 = pd.read_excel("DONNEE_LABEL_VRAI.xlsx")
# Votre prétraitement des données reste inchangé...
i = 4000
df = df.head(i)
# i est la ou je me suis arreté
# Renommer les colonnes par leur position
df = df.rename(columns={df.columns[0]: 'rue_init', df.columns[1]: 'PROP_LEV',df.columns[2]: 'LABEL'})
df = df.replace(np.nan, 0)
df['TYPE_VOIE'] =  df['rue_init'].str.split(' ').str[0]
df = df.replace(np.nan, 1)

df2 = df2.replace(np.nan, 1)
j = 1000
# i est la ou je me suis arreté
df2 = df2.head(j)
df2 = df2.rename(columns={df2.columns[0]: 'rue_init', df2.columns[1]: 'PROP_LEV',df2.columns[2]: 'LABEL'})

df2['TYPE_VOIE'] =  df2['rue_init'].str.split(' ').str[0]
df = pd.concat([df, df2], ignore_index=True)
df['rue_init'] = df['rue_init'].astype(str)
df['PROP_LEV'] = df['PROP_LEV'].astype(str)# Calculer des mesures de similarité
df["damerau_levenshtein"] = df.apply(lambda x: textdistance.damerau_levenshtein.normalized_similarity(x["rue_init"], x["PROP_LEV"]), axis=1)
df["jaro_winkler"] = df.apply(lambda x: textdistance.jaro_winkler(x["rue_init"], x["PROP_LEV"]), axis=1)
df["jaro"] = df.apply(lambda x: textdistance.jaro(x["rue_init"], x["PROP_LEV"]), axis=1)

# Calcul de la mesure de Sørensen-Dice pour chaque paire d'adresses
df['sorensen_dice'] = df.apply(lambda x: textdistance.sorensen_dice(x['rue_init'], x['PROP_LEV']), axis=1)

# Calcul de la plus longue sous-séquence commune (LCS) pour chaque paire d'adresses
df['lcs'] = df.apply(lambda x: textdistance.lcsstr.normalized_similarity(x['rue_init'], x['PROP_LEV']), axis=1)
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Préparer les données pour l'entraînement
X = df[["damerau_levenshtein",  "jaro_winkler", "jaro","sorensen_dice", "lcs"]]
y = df["LABEL"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=35)


# Commencer une expérience MLflow
with mlflow.start_run():
    
    # Paramètres expérimentaux
    i = 4000
    j = 1000
    test_size = 0.1
    random_state = 35
    
    # Log des paramètres
    mlflow.log_param("i", i)
    mlflow.log_param("j", j)
    mlflow.log_param("test_size", test_size)
    mlflow.log_param("random_state", random_state)
    
    # Préparation et entraînement du modèle comme avant
    # ...
    
    # Entraînement du modèle
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    
    # Prédictions et évaluation
    y_pred = model.predict(X_test)
    
    # Log des métriques
    report = classification_report(y_test, y_pred, output_dict=True)
    mlflow.log_metric("precision", report['weighted avg']['precision'])
    mlflow.log_metric("recall", report['weighted avg']['recall'])
    mlflow.log_metric("f1-score", report['weighted avg']['f1-score'])
    
    # Log du modèle
    mlflow.sklearn.log_model(model, "model")
    
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


: 

: 